In [ ]:
import pinocchio as pin
from pinocchio.utils import *

import plotly.express as px

In [ ]:
model = pin.buildModelFromUrdf("robot_utils/panda-model/panda_arm.urdf")
data = model.createData()


In [ ]:
q0 = np.array([1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0])
v0 = np.zeros(7)
a0 = np.ones(7)*10

model.gravity = pin.Motion.Zero()

lr = 1e-3
loss = []
for i in range(10000):

    _, _, _ = pin.computeRNEADerivatives(model,data,q0,v0,a0)

    loss.append(np.linalg.norm(data.tau))

    dtau_dq = data.dtau_dq # Derivatives of the ID w.r.t. the joint config vector
    dtau_dv = data.dtau_dv # Derivatives of the ID w.r.t. the joint velocity vector
    dtau_da = data.M # Derivatives of the ID w.r.t. the joint acceleration vector

    a0 -= 210*lr*dtau_da.T@data.tau;

    
print(a0)
px.line(loss)

In [ ]:
_, _, _ = pin.computeRNEADerivatives(model,data,np.array([1.0, -1.0, 0.5, -2.0, 1.0, 2.0, -0.5]),np.zeros(7),np.array([10, 10, 10, 10, 10, 10, 10]))
np.set_printoptions(precision=3, suppress=True)
print(dtau_da)


In [ ]:


q0 = np.array([1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0])
qd0 = q0*5
qdd0 = np.ones(7)*10

pin.forwardKinematics(model,data,qTest)
J = pin.computeJointJacobian(model,data,q0,7)
R_bloc = np.zeros((6,6))
R_bloc[0:3, 0:3] = data.oMi[7].rotation
R_bloc[3:, 3:] = data.oMi[7].rotation
J = R_bloc@J

print("Initial config: ", q0)
print("Initial velocity: ", J@qd0)

lr = 1e-3
loss = []
for i in range(2000):

    pin.forwardKinematics(model,data,qTest)

    pin.computeForwardKinematicsDerivatives(model, data, q0, qd0, qdd0)
    (dv_dq,dv_dv) = pin.getJointVelocityDerivatives(model,data,7,pin.ReferenceFrame.LOCAL)
    
    J = pin.computeJointJacobian(model,data,q0,7)
    R_bloc = np.zeros((6,6))
    R_bloc[0:3, 0:3] = data.oMi[7].rotation
    R_bloc[3:, 3:] = data.oMi[7].rotation
    # J = R_bloc@J

    loss.append(np.linalg.norm(J@qd0))

    q0 -= lr*dv_dq.T@(J@qd0)

    
print("Final config: ", q0)
print("Final velocity: ", J@qd0)
px.line(loss)


In [ ]:
import pinocchio as pin
import numpy as np

model = pin.buildModelFromUrdf("robot_utils/panda-model/panda_arm.urdf")
data = model.createData()

qTest = np.array([0.0, 0.0, 0.0, -1.5, 0.0, 1.0, 0.0])
qd = np.array([1, 1, 1, 1, 1, 1, 1])

pin.forwardKinematics(model,data,qTest)
# pin.computeJointJacobians(model,data,qTest)

J = pin.computeJointJacobian(model,data,qTest,7)

R_bloc = np.zeros((6,6))
R_bloc[0:3, 0:3] = data.oMi[7].rotation
R_bloc[3:, 3:] = data.oMi[7].rotation
print(R_bloc@J@qd)

In [ ]:
np.set_printoptions(precision=3, suppress=True)
print(R_bloc)
